### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Create a dictionary for the summary
district_summary = {}

# Get the total number of schools
district_summary["Total Schools"] = school_data["School ID"].count()

# Get the total number of students
total_students = student_data["Student ID"].count()
district_summary["Total Students"] = "{:,}".format(total_students)

# Calculate the total budget for all schools
total_budget = school_data["budget"].sum()
district_summary["Total Budget"] = "${:,.2f}".format(total_budget)

# Get the average math score
average_math = school_data_complete["math_score"].mean()
district_summary["Average Math Score"] = "{:.6f}".format(average_math)

# Calculate the average reading score
average_reading = school_data_complete["reading_score"].mean()
district_summary["Average Reading Score"] = "{:.6f}".format(average_reading)

# Create filter conditions for students with 70 or greater on math a& reading
# Record the number of students
math_passing = school_data_complete["math_score"] >= 70
reading_passing = school_data_complete["reading_score"] >= 70
all_student_count = school_data_complete["Student ID"].count()

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_df = school_data_complete.loc[math_passing]
percentage_math = passing_math_df["Student ID"].count() / all_student_count * 100
district_summary["% Passing Math"] = "{:.6f}".format(percentage_math)

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_df = school_data_complete.loc[reading_passing]
percentage_reading = passing_reading_df["Student ID"].count() / all_student_count * 100
district_summary["% Passing Reading"] = "{:.6f}".format(percentage_reading)

# Calculate the percentage of students who passed math and reading (% Overall Passing)
passing_both_df = school_data_complete.loc[math_passing & reading_passing]
percentage_both = passing_both_df["Student ID"].count() / all_student_count * 100
district_summary["% Overall Passing"] = "{:.6f}".format(percentage_both)

# Create a dataframe to hold results and display
district_df = pd.DataFrame([district_summary])
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Create a dataframe from school data
columns_needed = ["school_name", "type", "size", "budget"]
school_df = school_data[columns_needed].sort_values(["school_name"])

# Get Per Student Budget
school_df["Per Student Budget"] = school_df["budget"] / school_df["size"]

# Filter two columns with grouping by school names
score_group = school_data_complete.groupby("school_name")[["math_score", "reading_score"]]

# Get the average math and reading scores, then merge
school_group = score_group.mean()
school_df = pd.merge(school_df, school_group, on = "school_name", how = "outer")

# Get the passing student percentage of math and reading, then merge
school_group2 = score_group.apply(lambda x: (x >= 70).sum() / (x >= 0).sum() * 100)
school_df = pd.merge(school_df, school_group2, on = "school_name", how = "outer")

# Using apply to get student percentage (passing both), then merge
school_group3 = score_group.apply(
    lambda x: ((x["math_score"] >= 70) & (x["reading_score"] >= 70)).sum() / (x >= 0).sum() * 100)['math_score']
school_df = pd.merge(school_df, school_group3, on = "school_name", how = "outer")

# Rename columns
school_df = school_df.rename(columns = {
    "type": "School Type",
    "size": "Total Students",
    "budget": "Total School Budget",
    "math_score_x": "Average Math Score",
    "reading_score_x": "Average Reading Score",
    "math_score_y": "% Passing Math",
    "reading_score_y": "% Passing Reading",
    "math_score": "% Overall Passing"
})

# Set index to school name and re-index
school_df.set_index("school_name", drop = True, inplace = True)
school_df = school_df.rename_axis(None)

# Duplicate the dataframe to display formatted result 
school_df_display = school_df.copy()
school_df_display["Total School Budget"] = school_df_display["Total School Budget"].map("${:,.2f}".format)
school_df_display["Per Student Budget"] = school_df_display["Per Student Budget"].map("${:,.2f}".format)

school_df_display

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Sort values with first 5 rows
top_performing_school = school_df_display.sort_values("% Overall Passing", ascending = False).iloc[0:5]
top_performing_school

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# Sort values with first 5 rows
bottom_performing_school = school_df_display.sort_values("% Overall Passing", ascending = True).iloc[0:5]
bottom_performing_school

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Using conditional statement to get students of each grade level
nineth_grade = school_data_complete[school_data_complete["grade"] == "9th"]
nineth_grade_math = nineth_grade.groupby(["school_name"]).mean()["math_score"]

tenth_grade = school_data_complete[school_data_complete["grade"] == "10th"]
tenth_grade_math = tenth_grade.groupby(["school_name"]).mean()["math_score"]

eleventh_grade = school_data_complete[school_data_complete["grade"] == "11th"]
eleventh_grade_math = eleventh_grade.groupby(["school_name"]).mean()["math_score"]

twelfth_grade = school_data_complete[school_data_complete["grade"] == "12th"]
twelfth_grade_math = twelfth_grade.groupby(["school_name"]).mean()["math_score"]

# Combine each series into a new dataframe
math_grade_df = pd.DataFrame({
    "9th": nineth_grade_math, 
    "10th": tenth_grade_math,
    "11th": eleventh_grade_math,
    "12th": twelfth_grade_math})
math_grade_df = math_grade_df.rename_axis(None)

# Give the data a cleaner formatting
math_grade_df["9th"] = math_grade_df["9th"].map("{:.2f}".format)
math_grade_df["10th"] = math_grade_df["10th"].map("{:.2f}".format)
math_grade_df["11th"] = math_grade_df["11th"].map("{:.2f}".format)
math_grade_df["12th"] = math_grade_df["12th"].map("{:.2f}".format)

math_grade_df

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Using the conditional grade groups from above
# Get series for reading score for each grade level
nineth_grade_reading = nineth_grade.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading = tenth_grade.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading = eleventh_grade.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading = twelfth_grade.groupby(["school_name"]).mean()["reading_score"]

# Combine each series into a new dataframe
reading_grade_df = pd.DataFrame({
    "9th": nineth_grade_reading, 
    "10th": tenth_grade_reading,
    "11th": eleventh_grade_reading,
    "12th": twelfth_grade_reading})
reading_grade_df = reading_grade_df.rename_axis(None)

# Give the data a cleaner formatting
reading_grade_df["9th"] = reading_grade_df["9th"].map("{:.2f}".format)
reading_grade_df["10th"] = reading_grade_df["10th"].map("{:.2f}".format)
reading_grade_df["11th"] = reading_grade_df["11th"].map("{:.2f}".format)
reading_grade_df["12th"] = reading_grade_df["12th"].map("{:.2f}".format)

reading_grade_df

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Create a dataframe to get required columns
school_peform_df = school_df[
    ["Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"]]

# Create bins and labels for cut
bins = [0, 584, 629, 644, 675]
spending_ranges = ["<$584", "$585-629", "$630-644", "$645-675"]

# Cut by using bins/labels
school_peform_df["Spending Ranges (Per Student)"] = pd.cut(school_peform_df["Per Student Budget"], bins, labels = spending_ranges)

# Group by spending ranges and get the mean
school_perform_group = school_peform_df.groupby("Spending Ranges (Per Student)").mean()
school_perform_group.drop("Per Student Budget", axis = 1, inplace = True)

# Give the data a cleaner formatting
school_perform_group["Average Math Score"] = school_perform_group["Average Math Score"].map("{:.2f}".format)
school_perform_group["Average Reading Score"] = school_perform_group["Average Reading Score"].map("{:.2f}".format)
school_perform_group["% Passing Math"] = school_perform_group["% Passing Math"].map("{:.2f}".format)
school_perform_group["% Passing Reading"] = school_perform_group["% Passing Reading"].map("{:.2f}".format)
school_perform_group["% Overall Passing"] = school_perform_group["% Overall Passing"].map("{:.2f}".format)

school_perform_group

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Create a dataframe to get required columns
school_size_df = school_df[
    ["Total Students",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"]]

# Create bins and labels for cut
bins = [0, 999, 1999, 5000]
school_sizes = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Cut by using bins/labels
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], bins, labels = school_sizes)

# Group by school size and get the mean
school_size_df = school_size_df.groupby("School Size").mean()
school_size_df.drop("Total Students", axis = 1, inplace = True)

school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# Create a dataframe to get required columns
school_type_df = school_df[
    ["School Type",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"]]

# Group by school type and get the mean
school_type_df = school_type_df.groupby("School Type").mean()
school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Observable Trends

1. According to the report of Top/Bottom Performing Schools (By % Overall Passing), we can see that schools with higher overall passing rates (>90%) are all Charter schools. On the contrary, schools with lower overall passing rates (<54%) are all District schools.

2. From the report of Math/Reading Scores by Grade, we can see that the math grades have no significant relationship with the grade level of students in the same school.
1-top-schools

3. From the report of Scores by School Spending, as the spending budget per students increases (from $584/student to $675/student), the overall passing percentage of math and reading decreases (from 90% to 54%).
1-top-schools

4. From the result of Scores by School Size, when the school has students more than 2000, the overall passing percentage drops from around 90% to 58%.

5. From the result of Scores by School Type, Charter schools have much higher overall passing percentages than District schools.